In [1]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
from scipy.stats import poisson

In [5]:
a = np.linspace(0,99,100).reshape(10,10)
a

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
       [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
       [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
       [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.],
       [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.],
       [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.],
       [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.]])

In [7]:
np.average(a, axis=0)

array([45., 46., 47., 48., 49., 50., 51., 52., 53., 54.])

In [30]:
poisson.ppf(0.98, np.array([1.4,2.,3.,4.,5.]), loc=0)

array([ 4.,  5.,  7.,  9., 10.])

In [31]:
b = np.linspace(1,10,10)
b.shape

(10,)

In [10]:
def test():
    a = {}
    a["asdf"] = 1
    a["basdf"] = 2
    return a

In [18]:
b = test()
b.items()

dict_items([('asdf', 1), ('basdf', 2)])

In [13]:
for key, value in b.items:
    print(key, value)

TypeError: 'builtin_function_or_method' object is not iterable

In [2]:
a = np.linspace(1,10,10)
a

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [5]:
a[::-1]

array([10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.])

In [4]:
3 > None

TypeError: '>' not supported between instances of 'int' and 'NoneType'